In [128]:
import numpy as np
import IPython.display as ipd   # using ipd.Audio and ipd.Display to play sounds
from fractions import Fraction

fs = 48000
pi = np.pi

wav = lambda x : np.sin(2*pi*x)
env = lambda x : np.exp(-x)

In [181]:
#Scales

chromatic = [1, 16/15, 9/8, 6/5, 5/4, 4/3, 25/18, 3/2, 8/5, 5/3, 9/5, 15/8]

# modes of major/minor
major = [1, 9/8, 5/4, 4/3, 3/2, 5/3, 15/8]
dorian = [1, 9/8, 6/5, 4/3, 3/2, 5/3, 9/5]
phrygian = [1, 16/15, 6/5, 4/3, 3/2, 8/5, 9/5]
lydian = [1, 9/8, 5/4, 25/18, 3/2, 5/3, 15/8]
mixolydian = [1, 9/8, 5/4, 4/3, 3/2, 5/3, 9/5]
minor = [1, 9/8, 6/5, 4/3, 3/2, 8/5, 9/5]
locrian = [1, 16/15, 6/5, 4/3, 25/18, 8/5, 9/5]

# pentatonic and blues scales
major_pent = [1, 9/8, 5/4, 3/2, 5/3]
minor_pent = [1, 6/5, 4/3, 3/2, 9/5]
major_blues = [1, 9/8, 6/5, 5/4, 3/2, 5/3]
minor_blues = [1, 6/5, 4/3, 25/18, 3/2, 9/5]

harmonic_minor = [1, 9/8, 6/5, 4/3, 3/2, 8/5, 15/8]
melodic_minor = [1, 9/8, 6/5, 4/3, 3/2, 5/3, 15/8]
whole_tone = [1, 9/8, 5/4, 25/18, 8/5, 9/5]

def pitch(degree,scale,key):
    
    deg_oct = ( degree%len(scale), degree//len(scale) )
    pitch = key * scale[deg_oct[0]] * ( 2**deg_oct[1] )
        
    return pitch

In [48]:
def bank(f,long,wav_block,prints = False, graph = False):
    
    f_frac = Fraction(f).limit_denominator(20)
    cycle_frac = Fraction(fs,f_frac)
    cycle_samps = cycle_frac.numerator
    cycle_periods = cycle_frac.denominator
    
    max_samps = round(fs*long)
    unique_samps = min(max_samps,cycle_samps)
    
    samp_domain = np.arange(unique_samps)*float(f_frac)%fs
    
    ref_low = np.floor(samp_domain).astype(int)
    ref_high = (ref_low + 1)%fs
    decimal = samp_domain%1.0
    
    low_samps = np.take(wav_block,ref_low)
    high_samps = np.take(wav_block,ref_high)
    
    wav_chunk = low_samps*(1-decimal) + high_samps*decimal  
    
    if prints:
        print('\n f: %s \n' % f)
        print('f_frac: %s' % f_frac)
        print('samples per cycle: %s' % cycle_samps)
        print('periods per cycle: %s' % cycle_periods)
        print('max samps: %s' % max_samps)  
    
    if cycle_samps < max_samps:
        tiles = max_samps // cycle_samps + 1
        if prints:
            print('tiles: %s' % tiles)
        wav_chunk = np.tile(wav_chunk,tiles)[:max_samps]
           
    return wav_chunk

def initialize_wav_bank(Fs,longs,prints = False):
    
    wav_block = wav(np.arange(0,1,1/fs))
    wav_bank = [bank(f,l,wav_block,prints) for f,l in zip(Fs,longs)]
    
    return wav_bank

In [49]:
def initialize_env_block(FRL):
    
    max_L = max(FRL[:,2])
    env_domain = np.arange(0,max_L,1/fs)
    env_block = env(env_domain)
    
    return env_block

In [188]:
def process_ins(kwargs):
    
    keys = ['F','R','L','D','key','scale','tempo','prints']
    values = [None,None,None,[0],440,minor,120,False]
    ins = dict(zip(keys, values)) 
    
    for k in keys:
        if k in kwargs:
            ins[k] = kwargs[k]

    F,D,scale,key,tempo = ins['F'],ins['D'],ins['scale'],ins['key'],ins['tempo']
    
    if not ('F' in kwargs):
        if (not ('D' in kwargs)) and ('R' in kwargs):
            F = np.full(len(kwargs['R']),key)
        else:
            F = [pitch(d,scale,key) for d in D]
    
    num_notes = len(F)
    
    if not ('R' in kwargs):
        ins['R'] = np.arange(num_notes)
    
    if not ('L' in kwargs):
        ins['L'] = np.ones(num_notes)
        
    R = np.array(ins['R'])/tempo*60
    L = np.array(ins['L'])/tempo*60
        
    FRL = np.array(list(zip(F,R,L)))
        
    return FRL, ins['prints']

def create_out(FRL,wav_env_bank,F_key):

    starts = np.round(fs*FRL[:,1]).astype(int)
    durs = np.round(fs*FRL[:,2]).astype(int)
    ends = starts + durs
    
    total_duration = max(ends)
    out = np.zeros(total_duration)
    
    num_notes = FRL.shape[0]
    
    for k in range(num_notes):
        out[ starts[k] : ends[k] ] += wav_env_bank[F_key[k]][:durs[k]]
        
    return out

In [212]:
def main(**kwargs):
    
    FRL, prints = process_ins(kwargs)
    
    F_unique = np.sort(np.array(list(set(FRL[:,0]))))
    F_key = [np.where(F_unique == f)[0][0] for f in FRL[:,0]]
    
    long_note = np.array([max(FRL[FRL[:, 0] == f, 2]) for f in F_unique])
    
    wav_bank = initialize_wav_bank(F_unique,long_note,prints)
    env_block = initialize_env_block(FRL)
    
    long_durs = np.round(long_note*fs).astype(int)
    wav_env_bank = [env_block[:s]*w[:s] for w,s in zip(wav_bank,long_durs)]
    
    out = create_out(FRL,wav_env_bank,F_key)
    ipd.display(ipd.Audio(out, rate=fs))  

    return out

In [213]:
F1 = [800,700,600,500,400,300,200,100]*2
R1 = [0,0.1,0.2,0.3,0,0.1,0.3,0.4] + [1,1.1,1.2,1.3,1,1.1,1.3,1.4]

F2 = list(np.array(F1)*1.75)
R2 = list(np.array(R1)+2)

F3 = F1 + F2
R3 = R1 + R2

F4 = F3[::-1]
R4 = list(np.array(R3) + 4)

R5 = np.array([0]*5 + [0.4] + [0.1] + [0.35]) + 8
R6 = R5[::-1] + 1

R7 = list(R5) + list(R6)
R8 = R7 + list(np.array(R7) + 2)
R9 = R8 + list(np.array(R8) + 4)

R = (R3 + R4 + R9)[::-1]
F = (F3 + F4)*2
L = [1]*len(F)

In [214]:
d1 = [2,4,6]
d2 = [3,5,7]
d3 = [1,3,5]

da = d1 + d2 + d3 + d1
db = 3*d1 + 3*d2 + 3*d3 + 3*d1
d = da + db

r1 = np.repeat([0,3,6,9],3)
r2 = np.repeat([12,13,14],3)
r = np.concatenate((r1,r2,r2+3,r2+6,r2+9))

l1 = 3*np.ones(12)
l2 = np.ones(36)*0.8
l = np.concatenate((l1,l2))

In [216]:
out = main(D=d,R=r,L=l,scale = major,tempo = 480)